## Exploring Llama2 for Q&A given some context

In [1]:
import torch
import transformers
from transformers import LlamaTokenizer, AutoModelForCausalLM

/home/td/Documents/GitHub/Llama2-Chat/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# model path
model = "/home/td/Documents/Models/Llama-2-7b-chat-hf"

# set context
context = "Call graphs can be dynamic or static. A dynamic call graph is a record of an execution of the program, for example as output by a profiler. Thus, a dynamic call graph can be exact, but only describes one run of the program. A static call graph is a call graph intended to represent every possible run of the program. The exact static call graph is an undecidable problem, so static call graph algorithms are generally overapproximations. That is, every call relationship that occurs is represented in the graph, and possibly also some call relationships that would never occur in actual runs of the program."

# set prompt
prompt = f"Based on the following text generate a single question and five answer choices. Confirm the answer is directly in the given text. Provide the correct choice in your response.: \n{context}"

# load the tokenizer & model
tokenizer = LlamaTokenizer.from_pretrained(model)
model = AutoModelForCausalLM.from_pretrained(
        model,
        torch_dtype=torch.bfloat16,
        device_map="cuda:0",
        trust_remote_code=True,
    )

# create a pipeline
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.float16,
    device_map="cuda:0",
)

# create a sequence
sequences = pipeline(
    prompt,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    max_length=600,
)

for seq in sequences:
    print(f"Result: {seq['generated_text']}")